# Imports

In [7]:
import matplotlib.pyplot as plt
import tkinter
import numpy as np
import pandas as pd

# Inputs do usuário:

In [8]:
#conjunto de dados que vão poder ser inseridos pelo usuário
x = int(input("Qual a largura da placa?: "))#40 # comprimento da barra
y =  int(input("Qual a altura da placa?: ")) #40 #largura da barra
deltaX = int(input("Qual o espaçamento das divisões da largura da placa?: "))
deltaY= int(input("Qual o espaçamento das divisões da altura da placa?: "))
tempInicial =  int(input("Qual a temperatura inical dos pontos internos da placa da placa?: "))
deltaTempo = float(input("Uma iteração corresponde à quantos segundos? "))  #10**-3
ct = float(input("Qual a condutividade térmica do material?: ")) #0.23
ce = float(input("Qual o calor especifico do material?: ")) #897
density = float(input("Qual a densidade do material?: ")) #2.7 * 10**-6
alfa = ct/(ce*density) #9.4967 * 10**(-5)
isByTime = int(input("Deseja limitar o programa por tempo ou pelo maior erro admissível? (1 se por tempo e 0 se por erro) : "))
if isByTime:
    maxTimeCalculating = int(input("Qual o tempo limite de execução do programa?: "))#10 #in seconds (só deveria ser pedido para usuario, se o isByTime for True)
else :
    toleranciaAdmissivel = int(input("Qual o maior erro aceitável?: ")) #0.005 # só deveria ser pedido para usuario, se o isByTime for False
whichBorders =  input("Quais bordas possuem condições de contorno? (use 1 para a da esquerda, 2 para o topo, 3 para a da direita e 4 para a de baixo. Separe os números com virgula) :")
whichBorders = whichBorders.split(',')
newWhichBorders = [] # numbers from 1 to 4
for i in range(1,5):
    if(str(i) in whichBorders):
        newWhichBorders.append(i)
    else:
        newWhichBorders.append(None)
whichBorders = newWhichBorders
bordersFlowValue = []
bordersNameList = ['esquerda', 'superior', 'direita', 'inferior']
for i in whichBorders:
    if (i != None):
        temp = int(input("Qual a taxa de condução para a borda  {0}? ".format(bordersNameList[i-1])))
        bordersFlowValue.append(temp)
    else:
        bordersFlowValue.append(None)
temperaturesBorder= [None,None,None,None]
for i in range(0,4):
    temp  = int(input("Qual a temperatura inicial da borda {0}?".format(bordersNameList[i])))
    temperaturesBorder[i] = temp

Qual a largura da placa?: 40
Qual a altura da placa?: 40
Qual o espaçamento das divisões da largura da placa?: 8
Qual o espaçamento das divisões da altura da placa?: 8
Qual a temperatura inical dos pontos internos da placa da placa?: 0
Uma iteração corresponde à quantos segundos? 0.001
Qual a condutividade térmica do material?: 0.23
Qual o calor especifico do material?: 897
Qual a densidade do material?: 0.0000027
Deseja limitar o programa por tempo ou pelo maior erro admissível? (1 se por tempo e 0 se por erro) : 1
Qual o tempo limite de execução do programa?: 10
Quais bordas possuem condições de contorno? (use 1 para a da esquerda, 2 para o topo, 3 para a da direita e 4 para a de baixo. Separe os números com virgula) :1
Qual a taxa de condução para a borda  esquerda? 0
Qual a temperatura inicial da borda esquerda?0
Qual a temperatura inicial da borda superior?150
Qual a temperatura inicial da borda direita?50
Qual a temperatura inicial da borda inferior?0


# Cálculos

In [9]:
# tamanho de divisões na placa para cada eixo
xPointsLength = int(x/deltaX)
yPointsLength = int(y/deltaY)


def isIntern(i,j):
    localX = (i == 0) or (i == xPointsLength-1)
    localY = (j == 0) or (j == yPointsLength-1)
    return not (localX or localY)

def isVertix(i,j):
    return (i==0 and j==0) or (i==0 and j==yPointsLength-1)  or (i==xPointsLength-1 and j==0) or (i==xPointsLength-1 and j==yPointsLength-1)

# código que forma  a matriz no tempo 0
matrizDeTemp = []
tolerancias = []
print("Creating inital matrix...")
for i in range(0,xPointsLength):
    oordenadas = [];
    tols = []
    # existe essas condições pois as bordas possuem valores diferentes de temperatura
    for j in range(0, yPointsLength):
        if i == 0:
            oordenadas.append(temperaturesBorder[1])
            tols.append(0);
        elif i == (xPointsLength - 1):
            oordenadas.append(temperaturesBorder[3])
            tols.append(0);
        elif j == 0:
            oordenadas.append(temperaturesBorder[0])
            tols.append(0);
        elif j == (yPointsLength - 1):
            oordenadas.append(temperaturesBorder[2])
            tols.append(0);
        else:
            oordenadas.append(tempInicial)
            tols.append(float('inf'));
    matrizDeTemp.append(oordenadas)
    tolerancias.append(tols)
print("Created")

# cada interação do while é um  deltaTempo (segundos) que passou
Fo = alfa * deltaTempo / deltaX**2
print("Iterating though time...")
time = 0
condition = True #condição de parada do loop
while(condition):
    maiorTolerancia = 0
    for i in range(0, len(matrizDeTemp)):
        for j in range(0, len(matrizDeTemp[i])):
            tempvelha = matrizDeTemp[i][j]
            #aqui nessa parte tive de converter essa expressão para float128 pq eram números tão pequenos que não
            #cabiam em 64 bits
            if(isIntern(i,j)):
                matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]
            else: 
                if((j==0 and 1 in whichBorders) and (i != 0 and i != xPointsLength-1)):
                     matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + 2*deltaX*bordersFlowValue[0] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                elif((j==yPointsLength-1 and 3 in whichBorders) and (i != 0 and i != xPointsLength-1)):
                     matrizDeTemp[i][j] = np.float128(Fo * (2*deltaX*bordersFlowValue[2] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                elif((i==0 and 4 in whichBorders) and (j != 0 and j != yPointsLength-1)):
                     matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + 2*deltaY*bordersFlowValue[3])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                elif((i==xPointsLength-1 and 2 in whichBorders) and (j != 0 and j != yPointsLength-1)):
                     matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i-1][j] + matrizDeTemp[i-1][j] +  2*deltaY*bordersFlowValue[1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]
            if(not isVertix(i,j)):
                tempnova = matrizDeTemp[i][j]
                if(tempnova != 0):
                     erro = np.absolute((tempvelha - tempnova) / tempnova)
                else:
                    erro =np.absolute(tempvelha - tempnova)

                if(erro > maiorTolerancia):
                    maiorTolerancia = erro
            
    time+=1
    if(isByTime):
        if (time*deltaTempo > maxTimeCalculating):
            condition = False
    else:
        condition = maiorTolerancia > tolerânciaAdmissivel
    


print("Matrix answer: ")
print("\n")
print("Time elapsed: {0} segundos".format(time*deltaTempo))
print("Error: {0}".format(maiorTolerancia))


Creating inital matrix...
Created
Iterating though time...
Matrix answer: 


Time elapsed: 10.001 segundos
Error: 5.657584246855825e-10


# Tabela de Distribuição de temperatura ao longo da placa

In [12]:
minimun_value_scale = int (input ("Qual temperatura você considera extremamentr baixa para sua aplicação? "))
maximum_value_scale = int (input ("Qual temperatura você considera extremamente alta para sua aplicação? "))
diff = maximum_value_scale - minimun_value_scale
def colorChoices(val):
    color= None
    if val <= minimun_value_scale:
        color = '#36C'
    if val <= minimun_value_scale + 1*(diff/12):
        color = '#06C'
    if val <= minimun_value_scale + 2*(diff/12):
        color = '#3CF'
    elif val <= minimun_value_scale + 3*(diff/12):
        color = '#9FC'
    elif val <= minimun_value_scale + 4*(diff/12):
        color = '#9F9'
    elif val <= minimun_value_scale + 5*(diff/12):
        color = '#CF6'
    elif val <= minimun_value_scale + 6*(diff/12):
        color = '#00ff6b'
    elif val <= minimun_value_scale + 7*(diff/12):
        color = '#00ff04'
    elif val <= minimun_value_scale + 8*(diff/12):
        color = '#a3ff00'
    elif val <= minimun_value_scale + 9*(diff/12):
        color = '#fdff00'
    elif val <= minimun_value_scale + 10*(diff/12):
        color = '##ffc100'
    elif val <= minimun_value_scale + 11*(diff/12):
        color = '#ff7400'
    else:
        color = '#FF0000'
    return "background-color: {0}".format(color)
        
    
dataFrame = pd.DataFrame(matrizDeTemp)
dataFrameEstilizado = dataFrame.style.applymap(colorChoices)
dataFrameEstilizado


Qual temperatura você considera extremamentr baixa para sua aplicação? 0
Qual temperatura você considera extremamente alta para sua aplicação? 150


,0,1,2,3,4
0,150,150,150,150,150
1,40.6815,83.4758,94.7117,88.0167,50
2,29.2501,48.5102,57.3545,57.355,50
3,18.4901,23.9603,28.8409,34.049,50
4,0,0,0,0,0


# Gŕafico de tempo de execução X Tamanho da malha

In [ ]:
divisoes = []
tempos = []
isByTime = False
toleranciaAdmissivel = 0.005
for delta in range(1,10):
    # tamanho de divisões na placa para cada eixo
    xPointsLength = int(x/delta)
    yPointsLength = int(y/delta)
    print("Calculando de {0} em {1} cm".format(delta))

    # código que forma  a matriz no tempo 0
    matrizDeTemp = []
    tolerancias = []
    for i in range(0,xPointsLength):
        oordenadas = [];
        tols = []
        # existe essas condições pois as bordas possuem valores diferentes de temperatura
        for j in range(0, yPointsLength):
            if i == 0:
                oordenadas.append(temperaturesBorder[1])
                tols.append(0);
            elif i == (xPointsLength - 1):
                oordenadas.append(temperaturesBorder[3])
                tols.append(0);
            elif j == 0:
                oordenadas.append(temperaturesBorder[0])
                tols.append(0);
            elif j == (yPointsLength - 1):
                oordenadas.append(temperaturesBorder[2])
                tols.append(0);
            else:
                oordenadas.append(tempInicial)
                tols.append(float('inf'));
        matrizDeTemp.append(oordenadas)
        tolerancias.append(tols)

    # cada interação do while é um  deltaTempo (segundos) que passou
    Fo = alfa * deltaTempo / deltaX**2
    time = 0
    condition = True #condição de parada do loop
    while(condition):
        maiorTolerancia = 0
        for i in range(0, len(matrizDeTemp)):
            for j in range(0, len(matrizDeTemp[i])):
                tempvelha = matrizDeTemp[i][j]
                #aqui nessa parte tive de converter essa expressão para float128 pq eram números tão pequenos que não
                #cabiam em 64 bits
                if(isIntern(i,j)):
                    matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]
                else: 
                    if((j==0 and 1 in whichBorders) and (i != 0 and i != xPointsLength-1)):
                         matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + 2*deltaX*bordersFlowValue[0] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                    elif((j==yPointsLength-1 and 3 in whichBorders) and (i != 0 and i != xPointsLength-1)):
                         matrizDeTemp[i][j] = np.float128(Fo * (2*deltaX*bordersFlowValue[2] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                    elif((i==0 and 4 in whichBorders) and (j != 0 and j != yPointsLength-1)):
                         matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i+1][j] + matrizDeTemp[i-1][j] + matrizDeTemp[i][j+1] + 2*deltaY*bordersFlowValue[3])) + (1 - 4*Fo)*matrizDeTemp[i][j]

                    elif((i==xPointsLength-1 and 2 in whichBorders) and (j != 0 and j != yPointsLength-1)):
                         matrizDeTemp[i][j] = np.float128(Fo * (matrizDeTemp[i-1][j] + matrizDeTemp[i-1][j] +  2*deltaY*bordersFlowValue[1] + matrizDeTemp[i][j-1])) + (1 - 4*Fo)*matrizDeTemp[i][j]
                if(not isVertix(i,j)):
                    tempnova = matrizDeTemp[i][j]
                    if(tempnova != 0):
                         erro = np.absolute((tempvelha - tempnova) / tempnova)
                    else:
                        erro =np.absolute(tempvelha - tempnova)

                    if(erro > maiorTolerancia):
                        maiorTolerancia = erro

        time+=1
        if(isByTime):
            if (time*deltaTempo > maxTimeCalculating):
                condition = False
        else:
            condition = maiorTolerancia > toleranciaAdmissivel
    divisoes.append(delta)
    tempos.append(time*deltaTempo)
    
plt.plot(divisoes, tempos)
plt.title("Tamanho das divisões em CM X Tempo de execução")
plt.xlabel("Espaçamento em CM")
plt.ylabel("Tempo de execução (segundos)")
plt.show()
